In [1]:
## generate classes in datastructure package from xsd files(schema)

import os

package_name = "datastructure"
xsd_path = "taskSetInfo/xsd/"
if not os.path.exists(package_name):
    !xsdata {xsd_path} --recursive --package {package_name}

In [2]:
## initialize parser and set variables
from pathlib import Path
from datastructure import *
from xsdata.formats.dataclass.parsers import XmlParser
import shutil

# load the xml file
parser = XmlParser()

org_data_dir = "FMTV_model"
generated_data_dir = "generated_FMTV_json"

if os.path.exists(generated_data_dir):
    shutil.rmtree(generated_data_dir)
os.makedirs(generated_data_dir)

tasks_info = []

def find_task_info(tasks_info, name):
    for task_info in tasks_info:
        if task_info['task_name'] == name:
            return task_info

In [3]:
## parse core-processor mapping

import csv

mapping_xml = org_data_dir + "/mappingModel.xml"
mapping_model = parser.from_path(Path(mapping_xml), MappingModel)
    
# write task_name and core_index to csv file
for item in mapping_model.process_allocation:
    task_name = item.process.split("?")[0]
    core_index = int(item.scheduler.split("?")[0][-1])
    tasks_info.append({"task_name":task_name, "core_index":core_index})

In [4]:
## define functions to parse execution time of tasks

import numpy as np
import matplotlib.pyplot as plt


def findRunnableByName(runnables, runnable_name):
    return next((x for x in runnables if x.name == runnable_name), None)

def findLabelByName(labels, label_name):
    return next((x for x in labels if x.name == label_name), None)

def findStimuliModelByName(stimuls, stimul_name):
    return next((x for x in stimuls if x.name == stimul_name), None)

def getCycleForMemAccess(labels, label_name):
    number_bits = findLabelByName(labels, label_name).size.number_bits
    if number_bits % 32 == 0: # read 32 bits at a time
        number_access = int(number_bits / 32)
    else:
        number_access = int(number_bits / 32) + 1
    return number_access * 9 # 9 cycles to read/write 32 bits

def gamma_parameters(min_value, max_value, p_remain_promille, p1):    
    p2 = 1 - p_remain_promille
    if p1 > p2:
        (p1, p2) = (p2, p1)
        (min_value, max_value) = (max_value, min_value)
        
    alpha=(np.log(-np.log(p_remain_promille))-np.log(-np.log(1-p1)))/(np.log(max_value)-np.log(min_value))
    beta=min_value/(-np.log(1-p1))**(1/alpha)
    
    return (alpha, beta)

def show_hist_sample(sample):
    plt.hist(sample, bins=100, density=True)
    plt.show()

def samplingFromWeibullDistribution(num_samples, min_value, max_value, mean_value, p_remain_promille):
    prob_under_lower_bound = 0.1 # initial value
    average_state = 0 # 0: init, 1: mean > sample_mean, 2: mean < sample_mean
    while True:
        shape_p, scale_p = gamma_parameters(min_value, max_value, p_remain_promille, prob_under_lower_bound)
        sample = np.random.weibull(shape_p, num_samples) * scale_p
        #round to nearest integer and convert to int
        
        if min_value - np.min(sample) > 0:
            adj = min_value - np.min(sample)
            sample += adj
        sample = sample.clip(min_value, max_value)
        sample = np.rint(sample).astype(int)
        
        # break conditions
        if np.mean(sample) > mean_value:
            prob_under_lower_bound += 0.05 # if prob_under_lower_bound go high, sample_mean go low
            if average_state == 1: # prior_state = (mean > sample_mean)
                break
            else:
                average_state = 2 
        else:
            prob_under_lower_bound /= 2 # if prob_under_lower_bound go high, sample_mean go low
            if average_state == 2: # prior_state = (mean < sample_mean)
                break
            else:
                average_state = 1 
        
        if prob_under_lower_bound > 0.9 or prob_under_lower_bound < 0.0001:
            break
        
    # show_hist_sample(sample)
    return sample

In [5]:
population = samplingFromWeibullDistribution(100000, 12375, 213419, 82342, 0.005)


In [6]:
## parse execution time of tasks

sw_xml = org_data_dir + "/swModel.xml"
sw_model = parser.from_path(Path(sw_xml), SwModel)

stimulated_xml = org_data_dir + "/stimuliModel.xml"
stimulated_model = parser.from_path(Path(stimulated_xml), StimuliModel)

tasks = sw_model.tasks
runnables = sw_model.runnables
labels = sw_model.labels
stimulis = stimulated_model.stimuli

# variables for sampling
num_population = 10000
simulation_period_ms = 60000 # 60 sec

for task in tasks:
    # variable for sampling
    stimuli_name = task.stimuli.split("?")[0]
    task_stimuli = findStimuliModelByName(stimulis, stimuli_name)
    if task_stimuli.recurrence is not None: #periodic task
        recurrence_value = task_stimuli.recurrence.value
        recurrence_unit = task_stimuli.recurrence.unit
    else:
        stimulus_deviation = task_stimuli.stimulus_deviation
        assert stimulus_deviation is not None
        recurrence_value = stimulus_deviation.lower_bound.value
        recurrence_unit = stimulus_deviation.lower_bound.unit
        
    if recurrence_unit == "us":
        recurrence_value = recurrence_value / 1000
    else:
        assert recurrence_unit == "ms"
    num_samples = int(simulation_period_ms / recurrence_value)
    
    runnable_info = []
    for call in task.call_graph.graph_entries.calls:
        runnable_name = call.runnable.split("?")[0]
        # find runnable by name
        runnable = findRunnableByName(runnables, runnable_name)
        runnable_read_cycle = 0
        runnable_write_cycle = 0
        num_execution_item = 0
        for runnable_item in runnable.runnable_items:
            if runnable_item.access == "read":
                label_name = runnable_item.data.split("?")[0]
                runnable_read_cycle += getCycleForMemAccess(labels, label_name)
            elif runnable_item.access == "write":
                label_name = runnable_item.data.split("?")[0]
                runnable_write_cycle += getCycleForMemAccess(labels, label_name)
            else:
                num_execution_item += 1
                min_value = runnable_item.deviation.lower_bound.value
                max_value = runnable_item.deviation.upper_bound.value
                mean_value = runnable_item.deviation.distribution.mean.value
                p_remain_promille = runnable_item.deviation.distribution.p_remain_promille
                
                if num_population > num_samples:
                    population = samplingFromWeibullDistribution(num_population, min_value, max_value, mean_value, p_remain_promille)
                    sample = np.random.choice(population, num_samples)
                else:
                    sample = samplingFromWeibullDistribution(num_samples, min_value, max_value, mean_value, p_remain_promille)
        
        assert num_execution_item == 1
        
        # convert cycle to time (system clock = 200MHz, 1 cycle = 5ns[])
        runnable_read_time = runnable_read_cycle * 9
        runnable_write_time = runnable_write_cycle * 9
        runnable_execution_time = (sample).tolist()
        
        runnable_info.append({"read":runnable_read_time, "write":runnable_write_time, "execution":runnable_execution_time})
        
    task_info = find_task_info(tasks_info, task.name)
    task_info['priority'] = task.priority
    task_info['time_ns'] = runnable_info
    task_info['isRTTask'] = True

In [7]:
## parse period of periodic tasks and inter-arrival time of sporadic tasks

if tasks is None:
    sw_xml = org_data_dir + "/swModel.xml"
    sw_model = parser.from_path(Path(sw_xml), SwModel)
    tasks = sw_model.tasks

if stimulis is None:
    stimulated_xml = org_data_dir + "/stimuliModel.xml"
    stimulated_model = parser.from_path(Path(stimulated_xml), StimuliModel)
    stimulis = stimulated_model.stimuli

task_period_file_path = generated_data_dir + "/task_period.csv"

for task in tasks:
    task_name = task.name
    stimuli_name = task.stimuli.split("?")[0]
    task_stimuli = findStimuliModelByName(stimulis, stimuli_name)
    if task_stimuli.recurrence is not None: #periodic task
        period = task_stimuli.recurrence.value
        recurrence_unit = task_stimuli.recurrence.unit
        
        if recurrence_unit == "us":
            period = period * 1000 # convert to ns
        else:
            assert recurrence_unit == "ms"
            period = period * 1000000 # convert to ns
        
        task_info = find_task_info(tasks_info, task_name)
        task_info['isPeriodic'] = True
        task_info['period_ns'] = period
            
    else: #sporadic task
        stimulus_deviation = task_stimuli.stimulus_deviation
        assert stimulus_deviation is not None
        lower_bound = stimulus_deviation.lower_bound.value
        upper_bound = stimulus_deviation.upper_bound.value
        recurrence_unit = stimulus_deviation.lower_bound.unit
        
        if recurrence_unit == "us":
            lower_bound = lower_bound * 1000
            upper_bound = upper_bound * 1000
        else:
            assert recurrence_unit == "ms"
            lower_bound = lower_bound * 1000000
            upper_bound = upper_bound * 1000000
        
        task_info = find_task_info(tasks_info, task_name)
        task_info['isPeriodic'] = False # sporadic task
        task_info['lower_bound_ns'] = lower_bound
        task_info['upper_bound_ns'] = upper_bound
        

In [8]:
# merge runnable's execution time to make the phased task model

import os
import numpy as np

for task_info in tasks_info:
    phased_read_time = 0
    phased_write_time = 0
    num_samples = len(tasks_info[0]['time_ns'][0]['execution'])
    phased_execution_time = [0]* num_samples
    for runnable_info in task_info['time_ns']:
        phased_read_time += runnable_info['read']
        phased_write_time += runnable_info['write']
        phased_execution_time = [x + y for x, y in zip(phased_execution_time, runnable_info['execution'])]
        
    task_info['phased_read'] = phased_read_time
    task_info['phased_write'] = phased_write_time
    task_info['phased_execution'] = phased_execution_time


In [9]:
# insert Non_RT_Task
for i in range(4):
    Non_RT_task_info={}
    Non_RT_task_info['core_index'] = i
    Non_RT_task_info['task_name'] = "Non_RT_Task_1000ms_core" + str(Non_RT_task_info['core_index'])
    Non_RT_task_info['isRTTask'] = False
    Non_RT_task_info['isPeriodic'] = True
    Non_RT_task_info['period_ns'] = 1000000000
    Non_RT_task_info['phased_read'] = 0
    Non_RT_task_info['phased_write'] = 0
    num_samples = (int)(simulation_period_ms / Non_RT_task_info['period_ns'] * 1000000)
    Non_RT_task_info['phased_execution'] = [100000000]* num_samples # execution time is 100ms
    Non_RT_task_info['time_ns'] = []
    Non_RT_task_info['time_ns'].append(
                                        {'read':Non_RT_task_info['phased_read'], 
                                        'write':Non_RT_task_info['phased_write'], 
                                        'execution':Non_RT_task_info['phased_execution']}
                                        )
    tasks_info.append(Non_RT_task_info)

In [10]:
# if task name is Task_10ms or Task_20ms, remove it
# for task_info in tasks_info:
#     if task_info['task_name'] == "Task_10ms" or task_info['task_name'] == "Task_20ms":
#         tasks_info.remove(task_info)

In [11]:
import json

with open(generated_data_dir + "/tasks_info.json", "w") as f:
    json.dump(tasks_info, f, indent=4)


In [12]:
len(tasks_info)

25

In [13]:
temp = task_info = find_task_info(tasks_info, "Non_RT_Task_1000ms_core1")
temp

{'core_index': 1,
 'task_name': 'Non_RT_Task_1000ms_core1',
 'isRTTask': False,
 'isPeriodic': True,
 'period_ns': 1000000000,
 'phased_read': 0,
 'phased_write': 0,
 'phased_execution': [100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000,
  100000000],
 'time_ns': [{'read': 0,
   'writ